In [1]:
from flask import Flask, request, jsonify
import cv2
import numpy as np
import os
from keras.models import load_model
import time
import matplotlib.pyplot as plt
import logging
from flask_cors import CORS
import random
import base64

In [2]:
model_path = r'D:\app\models\action_recognition_model.h5'
detection = load_model(model_path)

In [3]:
def preprocess_image(img):
    img = cv2.resize(img, (112, 112))  
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

In [4]:
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

@app.route('/predict', methods=['POST'])

def prediction():
    actions = ['red', 'eat', 'howcanihelpyou', 'sandwich', 'dream', 'ketchup', 'drink', 'timewhat', 'whatdidyousay', 'sleep',
               'blue', 'no', 'bread', 'cereal', 'nothing', 'green', 'seeyoulater', 'howoldareyou', 'nicetomeetyou', 'whereisthebathroom',
               'yes', 'goodbye', 'hello', 'hi', 'howareyou', 'ok', 'thanks', 'fine', 'sorry', 'whatsup', 'A', 'B', 'C', 'D', 'E',
               'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

    try:
        if 'video' not in request.files:
            return jsonify({'error': 'No video file provided'}), 400
        
        video_file = request.files['video']
        
        video_file_path = os.path.join('temp_video.mp4')
        video_file.save(video_file_path)

        video_capture = cv2.VideoCapture(video_file_path)
        frames = []
        while video_capture.isOpened():
            ret, frame = video_capture.read()
            if not ret:
                break
            frames.append(frame)
        
        video_capture.release()
        os.remove(video_file_path)

        processed_frames = np.array([preprocess_image(frame) for frame in frames if frame is not None])
        
        if len(processed_frames) == 0:
            return jsonify({'error': 'No valid frames found'}), 400

        predictions = detection.predict(processed_frames)
        avg_prediction = np.mean(predictions, axis=0)
        predicted_label = actions[np.argmax(avg_prediction)]

        frame_list = [cv2.imencode('.jpg', frame)[1].tobytes() for frame in frames if frame is not None]
        frame_list_base64 = [base64.b64encode(frame).decode('utf-8') for frame in frame_list]

        return jsonify({'label': predicted_label, 'frames': frame_list_base64}), 200
    
    except Exception as e:
        logging.exception("Error during prediction")
        return jsonify({'error': str(e)}), 500

In [5]:
if __name__ == '__main__':
    try:
        logging.info("Starting Flask server...")
        app.run(debug=True, host='0.0.0.0', port=5000, use_reloader=False)
    except Exception as e:
        logging.error(f"Error starting Flask server: {e}")

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.10.18:5000
INFO:werkzeug:Press CTRL+C to quit
